<a href="https://colab.research.google.com/github/PieBot314/Python_Collab/blob/main/Lesson_18_GuptaArnav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 18: Hunting Exoplanets In Space - Data Normalisation

### Teacher-Student Activities

In the last class, we evaluated the Random Forest Classifier model through a concept called the **confusion matrix**. We also calculated the precision, recall and f1-score values. They were undefined. Based on these three parameters, we concluded that our model needs a lot of improvement because it did not classify the stars having a planet as `2` rather it labelled every star as `1`.

In this lesson, we will process the data before deploying a prediction model so that it can learn the properties of the different stars through the training dataset.

Now, there is no right approach to the data processing method. It is an iterative process. It comes through experience and domain knowledge. (*The term 'domain' means a particular field of industry or academics*). For e.g., if you are a banker, then you would have the knowledge of the finance field. Similarly, if you are an astrophysicist, then you would have the technical knowledge of astronomy, quantum mechanics, optics etc. So, based on the knowledge of a respective field, data should be processed before deploying a prediction model.

In this class, we will perform the following data processing exercises:

1. Data Normalisation

2. Fast Fourier Transformation

3. Oversampling

Finally, we will check if the Random Forest Classifier model is still providing the expected results. If not, then we will deploy a much stronger prediction model called XGBoost Classifier. Generally, it doesn't require a lot of data pre-processing but it requires heavy computation resources such as high RAM, CPU, GPU and at least 4 cores processor. Thanks to Google Colab notebook, we have a decent computation power to run the XGBoost Classifier algorithm.

Hence, the XGBoost Classifier model must be used only when the data processing is not helping a prediction model in making accurate predictions.

Let's run all the codes in the code cells that we have already covered in the previous classes and begin this class from the **Activity 1: Data Normalisation** section. You too run the code cells until the first activity.


---

#### Loading The Datasets

Create a Pandas DataFrame every time you start the Jupyter notebook.

Dataset links (don't click on them):

1. Train dataset 

   https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/kepler-exoplanets-dataset/exoTrain.csv

2. Test dataset 

   https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/kepler-exoplanets-dataset/exoTest.csv

In [ ]:
# Load both the training and test datasets.
import numpy as np
import pandas as pd

exo_train_df = pd.read_csv('https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/kepler-exoplanets-dataset/exoTrain.csv')
exo_test_df = pd.read_csv('https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/kepler-exoplanets-dataset/exoTest.csv')

In [ ]:
# The shapes of the 'exo_train_df' and 'exo_test_df' DataFrames.
print(exo_train_df.shape)
exo_test_df.shape

(5087, 3198)


(570, 3198)

In the previous classes, we have already checked the datasets don't have a missing value. So, we can skip that part. 

---

#### Activity 1: Data Normalisation^^^

After creating a DataFrame and inspecting data for the missing values, we can normalise the data.

**What is data normalisation?**

Data normalisation is a process of standardising data. It brings every single data-point on a uniform scale. If you look at both the `exo_train_df` and `exo_test_df` DataFrames, they contain highly varying `FLUX` values. 

You can get the data summary using the `describe()` function to look at the variation of the data. 

**Syntax:** `dataframe_name.describe()`

In [ ]:
# Teacher Action: Get the data description by calling the 'describe()' function.
exo_test_df.describe()

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,FLUX.11,FLUX.12,FLUX.13,FLUX.14,FLUX.15,FLUX.16,FLUX.17,FLUX.18,FLUX.19,FLUX.20,FLUX.21,FLUX.22,FLUX.23,FLUX.24,FLUX.25,FLUX.26,FLUX.27,FLUX.28,FLUX.29,FLUX.30,FLUX.31,FLUX.32,FLUX.33,FLUX.34,FLUX.35,FLUX.36,FLUX.37,FLUX.38,FLUX.39,...,FLUX.3158,FLUX.3159,FLUX.3160,FLUX.3161,FLUX.3162,FLUX.3163,FLUX.3164,FLUX.3165,FLUX.3166,FLUX.3167,FLUX.3168,FLUX.3169,FLUX.3170,FLUX.3171,FLUX.3172,FLUX.3173,FLUX.3174,FLUX.3175,FLUX.3176,FLUX.3177,FLUX.3178,FLUX.3179,FLUX.3180,FLUX.3181,FLUX.3182,FLUX.3183,FLUX.3184,FLUX.3185,FLUX.3186,FLUX.3187,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
count,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,...,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000
mean,1.008772,515.411351,738.046404,532.603246,739.618088,530.949807,729.591491,517.421404,720.723544,481.015211,645.387632,394.718772,546.938491,345.028912,504.467965,380.654123,520.047825,441.260123,579.646895,508.071561,601.530474,549.411211,596.859053,553.084140,569.636439,528.500316,548.358895,553.495579,535.006789,595.535561,525.173000,610.541614,543.721965,514.244982,637.794386,500.698982,613.286561,486.848772,625.863018,495.518474,...,187.376035,7.543877,161.156421,190.397123,-2.452316,205.491807,0.935298,234.495684,-51.111298,185.505404,-120.917772,131.590105,-142.733702,135.401070,197.012544,-43.925544,255.463053,-4.608421,290.270912,10.029298,305.127333,15.238719,312.386632,19.858789,322.278333,314.102702,32.335509,298.475789,24.615982,293.036246,60.027754,307.682825,88.976842,301.299895,105.121684,291.509561,256.656789,121.810035,224.806035,133.954544
std,0.093329,12592.950138,12622.940170,12545.065255,12591.933126,12512.976544,12572.187328,12454.094432,12525.496793,12397.610716,12517.223250,12517.877770,12768.358252,12778.510980,12855.982377,12694.016374,12612.108328,12522.478768,12484.462623,12466.285290,12483.420709,12537.360038,12548.009282,12540.103559,12556.259436,12531.238889,12502.645118,12456.197192,12418.404486,12438.343012,12462.213244,12518.850363,12499.863959,12592.424871,12548.434477,12538.331244,12492.371136,12482.779563,12467.736551,12420.519819,...,7463.168362,7505.522973,7392.411838,7623.632456,7776.998243,7631.890848,7823.088147,7875.733743,8260.578016,8550.446124,9098.501182,9298.305738,9356.535627,9364.853475,8829.825279,8662.334286,8685.867568,8694.661885,8677.706409,8969.466550,8854.767277,9151.997154,9097.560566,9439.364369,9370.799546,9434.997132,9561.566345,9607.213463,9599.779262,9651.051209,9528.594808,9588.586341,9497.373179,9611.024800,9611.076529,9654.581767,9362.719825,9487.642217,9483.948248,9700.731923
min,1.000000,-59421.000000,-14663.430000,-56580.000000,-10817.590000,-54545.000000,-7744.500000,-53580.000000,-6373.400000,-53832.000000,-33382.160000,-53675.330000,-73021.910000,-87191.970000,-90995.410000,-83457.980000,-67208.450000,-47678.560000,-28128.720000,-23532.000000,-6869.240000,-18196.000000,-6830.550000,-7814.000000,-12776.370000,-15675.070000,-12110.820000,-7167.510000,-9098.000000,-7297.800000,-21951.000000,-7605.660000,-26387.000000,-34668.000000,-9901.930000,-36912.000000,-11147.320000,-36700.00

As you can see, the values in the `FLUX.1` column range between `-227,856.3` (minimum `FLUX.1` value) and `1,439,240` (maximum `FLUX.1` value). 

**Note:** `-2.278563e+05` is equivalent to $-2.278563\times10^5$ and `1.439240e+06` equivalent to $1.439240\times10^6$ 

In the `FLUX.1` column, the difference in the maximum and minimum values, i.e., 

$$1,439,240 - (-227,856.3) = 1,667,096.3$$


is huge because of the $10^5$ and $10^6$ scales. 

Similarly, the data in all the other `FLUX` columns also vary a lot because they lie on a huge scale. The big figures are less readable. For e.g., `122` (one hundred twenty two) is more readable than `1,439,240` (one million, four hundred thirty nine thousand, two hundred forty).

The data normalisation process lowers the scale and brings all the data-points on the same scale.

**Why must data be normalised?**

The machine learning models are quite sensitive to the scale of data. They give more importance to the larger values while learning the properties of data. Hence, it becomes crucial for us to remove this bias by bringing down all the data-points on the same scale.


**How to normalise data?**

There are various methods of data normalisation. We will cover all of them throughout this course whenever we need them. For the time being, we will use the *mean normalisation* method. Let's understand the *mean normalisation* method.

Consider a series of numbers having the values 

$$x_1, x_2, x_3, ... , x_N$$ 

where $N$ is the total number of values in a series.

Let 

- $x_{mean}$ denote the mean (or average) value of a series

- $x_{min}$ denote the minimum value in a series and 

- $x_{max}$ denote the maximum value in a series

The normalised value in a series is calculated as

$$x_{norm} = \frac{x_p - x_{mean}}{x_{max} - x_{min}}$$

where 

$$x_p = x_1, x_2, x_3, ..., x_N$$

So after normalisation, the new values in the series will be

$$\left(\frac{x_1 - x_{mean}}{x_{max} - x_{min}}\right), \left(\frac{x_2 - x_{mean}}{x_{max} - x_{min}}\right), \left(\frac{x_3 - x_{mean}}{x_{max} - x_{min}}\right), ..., \left(\frac{x_N - x_{mean}}{x_{max} - x_{min}}\right)$$

For e.g., consider the series $[5, 192, 20019, 12, 209]$.

- The average value of the series is $x_{mean} = 4087.4$

- The minimum value in the series is $x_{min} = 5$

- The maximum value in the series is $x_{max} = 20019$

So, after normalisation, the new series would have the following numbers.

$$\left[ \left( \frac{5 - 4087.4}{20019 - 5} \right), \left( \frac{192 - 4087.4}{20019 - 5} \right), \left( \frac{20019 - 4087.4}{20019 - 5} \right), \left( \frac{12 - 4087.4}{20019 - 5} \right), \left( \frac{209 - 4087.4}{20019 - 5} \right) \right]$$

$$\Rightarrow \left[-0.203977, -0.194634, 0.796023, -0.203627, -0.193784 \right]$$

or 

$$\left[-\frac{203,977}{1,000,000}, -\frac{194,634}{1,000,000}, \frac{796,023}{1,000,000}, -\frac{203,627}{1,000,000}, -
\frac{193,784}{1,000,000} \right]$$

As you can see, after normalisation all the new values are on the same scale which is $\frac{1}{1,000,000}$ or $10^{-6}$.

So, now let's create a function which normalises data in a series. It should take a Pandas series as an input and should return a normalised series as an output.





In [ ]:
# Student Action: Create a function to normalise a Pandas series using the mean normalisation method.
def mean_normalisation(series):
  return (series-series.mean())/(series.max()-series.min())


Now, let's test the `mean_normalise()` function on the $[5, 192, 20019, 12, 209]$ series. If we get the desired output, then it means the function is working correctly.

In [ ]:
# Student Action: Test the 'mean_normalise()' function on the '[5, 192, 20019, 12, 209]' series.

mean_normalisation(pd.Series([5, 192, 20019, 12, 209]))

0   -0.203977
1   -0.194634
2    0.796023
3   -0.203627
4   -0.193784
dtype: float64

Now, let's apply the `mean_normalise()` function on the `exo_train_df` DataFrame to normalise only the `FLUX` values.

Using the `iloc[]` function, we will first exclude the `LABEL` column from the DataFrame and then will apply the `mean_normalise()` function on the `exo_train_df` DataFrame using the `apply()` function.

The `apply()` function takes two inputs. 

- The first input is the function that needs to be applied which in this case is `mean_normalise()` function. 

- The second input is the `axis` value which signifies whether the function in the first input needs to be applied vertically or horizontally.

**Syntax:** `dataframe.apply(function_name, axis)`

**Note:** Whenever you apply a function, say `function_name()` on a DataFrame using the `apply()` function, remove the brackets from the name of the function (i.e., `function_name`) to be applied. That's why the syntax is `dataframe.apply(function_name, axis)`

A DataFrame has two axes (axes is plural of axis). 

- The first axis is the vertical axis. It is represented as `axis = 0` 

- The second axis is the horizontal axis. It is represented as `axis = 1`

The DataFrame axes define whether an operation needs to be applied row-wise or column-wise. Refer to the image shown below.

<img src='https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/images/lesson-16/dataframe-axes.png' width=600>

1. If `axis = 0`, then it means the function needs to be applied **vertically**. In other words, the function will be applied on all the rows but only **one column at a time.** So, on the `exo_train_df` DataFrame, if the `mean_normalise()` function is applied **vertically**, then it will be applied in the following order:

    - `exo_train_df.iloc[:, 0]`, i.e., all the rows and the `FLUX.1` column at a time.
    
    - `exo_train_df.iloc[:, 1]`, i.e., all the rows and the `FLUX.2` column at a time.

    - `exo_train_df.iloc[:, 2]`, i.e., all the rows and the `FLUX.3` column at a time.

    ...

    - `exo_train_df.iloc[:, 3197]`, i.e., all the rows and the `FLUX.3197` column at a time.

2. If `axis = 1`, then it means the function needs to be applied **horizontally**. This means the function will be applied on all the columns but only **one row at a time.** So, on the `exo_train_df` DataFrame, if the `mean_normalise()` function is applied **horizontally**, then it will be applied in the following order:

    - `exo_train_df.iloc[0, :]`, i.e., the first row and all the columns at a time.
    
    - `exo_train_df.iloc[1, :]`, i.e., the second row and all the columns at a time.

    - `exo_train_df.iloc[2, :]`, i.e., the third row and all the columns at a time.

      ...

    - `exo_train_df.iloc[5086, :]`, i.e., the last row and all the columns at a time.

We will apply the `mean_normalise()` function **horizontally** on the `exo_train_df` DataFrame to normalise the `FLUX` values for a star at a time.


In [ ]:
# Teacher Action: Apply the 'mean_normalise' function horizontally on the training DataFrame. 
norm_train_df = exo_train_df.iloc[:,1:].apply(mean_normalisation, axis=1)
# After applying the 'mean_normalise' function on the 'exo_train_df' DataFrame, let's print the first 5 rows of the new DataFrame.
print(norm_train_df)

        FLUX.1    FLUX.2    FLUX.3  ...  FLUX.3195  FLUX.3196  FLUX.3197
0     0.053834  0.047391  0.006510  ...   0.033024  -0.003127  -0.031759
1    -0.050411 -0.042317 -0.081922  ...   0.022260   0.037550   0.043849
2     0.243983  0.245509  0.235186  ...  -0.017259  -0.036384  -0.048782
3     0.518501  0.551177  0.480659  ...  -0.019827  -0.019889   0.029163
4    -0.399904 -0.401872 -0.404199  ...  -0.135528  -0.145458  -0.181590
...        ...       ...       ...  ...        ...        ...        ...
5082 -0.191318 -0.193261 -0.167219  ...  -0.054037  -0.016964   0.012450
5083  0.419089  0.376920  0.384402  ...   0.039222  -0.023097   0.011590
5084  0.132006  0.134256  0.126314  ...   0.041723   0.037159   0.037335
5085  0.005570  0.002376 -0.007557  ...  -0.028346  -0.013317  -0.006191
5086  0.219815  0.208122  0.199000  ...  -0.015145  -0.013327   0.015635

[5087 rows x 3197 columns]


You can see that, all the data-points are on the same scale after mean normalisation. Notice that we didn't normalise the `LABEL` data as we intended.

Now, let's insert the `LABEL` column to the `norm_train_df` DataFrame to get the full DataFrame with the normalised `FLUX` values.

We can obtain the `LABEL` column from the `exo_train_df` DataFrame using the `exo_train_df['LABEL']` method.

To insert a column in a DataFrame, use the `insert()` function. It takes three inputs.

- The first input should be the desired column index of the new column after its insertion.

- The second input should be the desired column name.

- The third input should be the values of the new column.

**Syntax:** `dataframe.insert(loc=column_index, column=column_name, value=some_pandas_series)`

In [ ]:
# Teacher Action: Apply the 'insert()' function to add the 'LABEL' column to the 'norm_train_df' DataFrame.
norm_train_df.insert(loc=0,column='LABEL', value=exo_train_df['LABEL'])
# After inserting the 'LABEL' column to the 'norm_train_df' DataFrame, print its first five rows.
norm_train_df.head()

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,FLUX.11,FLUX.12,FLUX.13,FLUX.14,FLUX.15,FLUX.16,FLUX.17,FLUX.18,FLUX.19,FLUX.20,FLUX.21,FLUX.22,FLUX.23,FLUX.24,FLUX.25,FLUX.26,FLUX.27,FLUX.28,FLUX.29,FLUX.30,FLUX.31,FLUX.32,FLUX.33,FLUX.34,FLUX.35,FLUX.36,FLUX.37,FLUX.38,FLUX.39,...,FLUX.3158,FLUX.3159,FLUX.3160,FLUX.3161,FLUX.3162,FLUX.3163,FLUX.3164,FLUX.3165,FLUX.3166,FLUX.3167,FLUX.3168,FLUX.3169,FLUX.3170,FLUX.3171,FLUX.3172,FLUX.3173,FLUX.3174,FLUX.3175,FLUX.3176,FLUX.3177,FLUX.3178,FLUX.3179,FLUX.3180,FLUX.3181,FLUX.3182,FLUX.3183,FLUX.3184,FLUX.3185,FLUX.3186,FLUX.3187,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,0.053834,0.047391,0.006510,-0.023699,-0.031772,-0.086410,-0.093128,-0.068161,-0.057650,-0.109164,-0.139515,-0.105769,-0.117852,-0.100430,-0.083555,-0.072383,-0.069726,-0.037457,-0.061937,-0.006926,-0.022974,-0.049629,-0.029840,-0.062219,-0.069091,-0.090825,-0.092884,-0.126360,-0.163160,-0.151251,-0.153715,-0.169038,-0.099904,-0.099904,0.013165,-0.117647,-0.126739,-0.130153,-0.123659,...,-0.113989,-0.042873,-0.001536,0.017611,-0.058446,-0.019285,-0.025855,-0.212030,-0.212030,-0.191009,-0.231774,-0.051349,-0.131090,-0.074706,-0.015884,0.034122,0.077049,0.042656,0.078249,-0.130332,-0.130332,-0.063663,-0.017655,-0.017494,0.074220,0.037805,0.057786,-0.012547,-0.022679,-0.016981,-0.056482,-0.071934,-0.071934,0.009738,0.024779,0.052993,0.018843,0.033024,-0.003127,-0.031759
1,2,-0.050411,-0.042317,-0.081922,-0.052351,-0.115212,-0.104794,-0.126816,-0.124861,-0.122681,-0.105708,-0.126752,-0.108256,-0.105339,-0.126143,-0.110821,-0.086298,-0.047767,-0.065878,-0.037717,-0.001141,0.003026,0.037214,0.085169,0.028911,0.077075,0.038672,0.036236,0.031107,-0.314584,0.042567,0.042567,0.042567,0.042567,0.042567,0.039185,-0.001718,-0.010325,-0.034736,-0.043054,...,-0.046997,-0.012922,-0.009316,0.044699,0.009549,0.011264,0.056463,-0.081377,-0.081377,-0.103560,-0.081073,-0.037653,-0.009604,-0.010502,0.003202,0.025866,0.014790,0.070552,-0.003802,-0.019397,-0.019397,-0.027075,-0.026369,-0.041099,-0.002568,0.017916,0.030514,-0.008386,0.002786,0.031988,0.006648,-0.039721,-0.039721,-0.027988,0.004116,0.013124,-0.006847,0.022260,0.037550,0.043849
2,2,0.243983,0.245509,0.235186,0.227365,0.208538,0.212981,0.212283,0.222467,0.199285,0.221536,0.214684,0.211260,0.225183,0.201444,0.220969,0.220089,0.245225,0.254078,0.258014,0.247472,0.264313,0.252836,0.255171,0.252459,0.264773,0.257679,0.265071,0.243066,0.253580,0.246630,0.241438,0.243918,0.123501,0.123501,0.132215,0.135029,0.113722,0.104855,0.075933,...,-0.027578,-0.019301,-0.032434,-0.045498,-0.035989,-0.044037,-0.069363,-0.042981,-0.042981,-0.045256,-0.029601,-0.032211,-0.017380,-0.033490,-0.030611,-0.002554,-0.018348,-0.025717,-0.006848,0.002703,0.002703,-0.015463,-0.020083,-0.018734,-0.011746,-0.016729,-0.000153,-0.036733,-0.033788,-0.042809,-0.037161,0.002382,0.002382,-0.017715,-0.013523,-0.001456,-0.009299,-0.017259,-0.036384,-0.048782
3,2,0.518501,0.551177,0.480659,0.474051,0.504754,0.496863,0.511941,0.494687,0.496425,0.513506,0.494421,0.517984,0.497505,0.498726,0.505099,0.525390,0.541329,0.571828,0.587031,0.586123,0.538041,0.533719,0.507212,0.510735,0.489536,0.472000,0.444632,0.439058,0.460806,0.447748,0.454073,0.448593,0.146229,0.146229,0.113287,0.123856,0.115839,0.127065,0.132937,...,0.002969,0.029649,-0.014926,-0.032086,-0.013752,-0.015803,0.010782,-0.041371,-0.041371,-0.071432,-0.054131,-0.073091,-0.021737,0.004159,0.023605,0.004191,-0.016711,-0.029331,-0.036110,-0.049277,-0.049277,-0.017228,0.017656,0.023965,-0.006143,-0.005704,-0.027013,-0.033119,-0.039476,0.018892,0.016215,0.001435,0.001435,0.054324,0.038636,-0.012562,-0.006456,-0.019827,-0.019889,0.029163
4,2,-0.399904,-0.401872,-0.404199,-0.395473,-0.381734,-0.373293,-0.360070,-0.368986,-0.356861,-0.350022,-0.336272,-0.320243,-0.319939,-0.307239,-0.287833,-0.270844,-0.257219,-0.232375,-0.216411,-0.205402,-0.187382,-0.174118,-0.162633,-0.158

Now, you normalise the `FLUX` values in the `exo_test_df` DataFrame using the `mean_normalise()` function. Make sure that you apply the function horizontally to normalise the `FLUX` values for a star at a time.

In [ ]:
# Student Action: Apply the 'mean_normalize()' function on the testing DataFrame. Store the new DataFrame in the 'norm_test_df' variable.
norm_test_df = exo_test_df.iloc[:, 1:].apply(mean_normalisation, axis=1)
# After applying, the 'mean_normalise()' function on the 'exo_test_df' DataFrame, print the first 5 rows of the new DataFrame.
norm_test_df.head()

,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,FLUX.11,FLUX.12,FLUX.13,FLUX.14,FLUX.15,FLUX.16,FLUX.17,FLUX.18,FLUX.19,FLUX.20,FLUX.21,FLUX.22,FLUX.23,FLUX.24,FLUX.25,FLUX.26,FLUX.27,FLUX.28,FLUX.29,FLUX.30,FLUX.31,FLUX.32,FLUX.33,FLUX.34,FLUX.35,FLUX.36,FLUX.37,FLUX.38,FLUX.39,FLUX.40,...,FLUX.3158,FLUX.3159,FLUX.3160,FLUX.3161,FLUX.3162,FLUX.3163,FLUX.3164,FLUX.3165,FLUX.3166,FLUX.3167,FLUX.3168,FLUX.3169,FLUX.3170,FLUX.3171,FLUX.3172,FLUX.3173,FLUX.3174,FLUX.3175,FLUX.3176,FLUX.3177,FLUX.3178,FLUX.3179,FLUX.3180,FLUX.3181,FLUX.3182,FLUX.3183,FLUX.3184,FLUX.3185,FLUX.3186,FLUX.3187,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,0.273347,0.228221,0.196676,0.110003,0.104130,0.088690,0.040926,0.014337,0.013534,-0.052079,-0.054832,-0.068804,-0.079655,-0.016015,-0.066188,-0.064169,-0.075732,-0.029872,-0.039691,-0.056415,-0.014730,0.019521,0.079812,0.184609,0.265570,0.305717,0.340084,0.397851,0.429097,0.492255,0.564521,-0.131709,-0.131709,-0.131709,-0.121087,-0.073988,-0.073140,-0.113012,-0.090988,-0.061462,...,-0.007527,0.026450,-0.017873,-0.055658,0.004151,0.067630,0.087497,-0.054121,-0.054121,-0.011702,0.003531,-0.039003,-0.072428,-0.018217,-0.011473,0.029524,0.034158,0.071645,0.112779,0.197754,0.197754,0.096949,0.110186,0.050676,0.073549,0.068892,0.031703,0.023376,0.050355,0.011882,0.031635,0.042578,0.031451,-0.005393,0.028904,0.102708,0.071576,0.080408,0.616438,0.130742
1,0.394038,0.391480,0.392680,0.390974,0.388955,0.386673,0.386340,0.382364,0.381035,0.374634,0.365595,0.355955,0.344779,0.326513,0.302050,0.288028,0.267479,0.245166,0.225696,0.204367,0.188959,0.166178,0.151080,0.129661,0.115564,0.104075,0.103383,0.102889,0.109362,0.131452,0.162766,0.195175,0.262094,0.262094,0.254659,0.234561,0.217038,0.197559,0.181571,0.166160,...,-0.249097,-0.321740,-0.356853,-0.260644,-0.169015,-0.194461,-0.203200,-0.047612,-0.047612,-0.059527,-0.082817,-0.103659,-0.128578,-0.147432,-0.171446,-0.182454,-0.195212,-0.200677,-0.197563,0.074878,0.074878,0.088953,0.093556,0.088384,0.073554,0.071405,0.043269,0.022604,-0.002679,-0.018935,-0.047311,-0.075404,-0.092643,-0.118456,-0.134109,-0.150638,-0.164944,-0.171944,-0.166961,-0.148790
2,0.648150,0.627582,0.591444,0.519002,0.466046,0.385214,0.340496,0.281192,0.162553,0.119260,0.069748,-0.010734,-0.022744,-0.045385,-0.006923,0.017249,-0.014986,0.051756,0.045126,0.053615,0.059758,0.081073,0.055856,0.015809,-0.007906,-0.107775,-0.171644,-0.239483,-0.306187,-0.338697,-0.107417,-0.107417,-0.107417,-0.107417,-0.122133,-0.102014,-0.068872,-0.013302,0.014956,0.056107,...,-0.022256,-0.048845,-0.022302,-0.038442,-0.052251,-0.083176,-0.002191,-0.024047,-0.024047,-0.023925,-0.074831,-0.072713,-0.077209,-0.079091,-0.091627,-0.125196,-0.085210,-0.020016,-0.048883,0.244109,0.244109,0.244109,0.118688,0.046208,0.014201,-0.097274,-0.054964,-0.035257,-0.001916,-0.010436,0.018179,-0.034769,-0.032201,-0.041117,-0.057967,-0.128412,-0.067972,-0.119374,-0.023437,0.027941
3,-0.232813,-0.233212,-0.238944,-0.235869,-0.208281,-0.220224,-0.222214,-0.208586,-0.197319,-0.188186,-0.196253,-0.200681,-0.173424,-0.181969,-0.185891,-0.183767,-0.184645,-0.182481,-0.186111,-0.177627,-0.188482,-0.170188,-0.154902,-0.142609,-0.145010,-0.157352,-0.143322,-0.143334,-0.130733,-0.128905,-0.111279,-0.097154,-0.081508,-0.068529,-0.067585,-0.071967,-0.063690,-0.062919,-0.044958,-0.036084,...,0.023932,-0.373173,-0.373173,-0.373173,-0.373173,-0.373173,-0.373173,-0.373173,-0.373173,0.014351,0.038435,0.021458,0.032712,0.049598,0.046797,0.033132,0.025102,0.028333,0.007686,0.008762,-0.002312,-0.044482,-0.066509,-0.084833,-0.083919,-0.062776,-0.055758,-0.051376,-0.020587,0.008095,0.056186,0.047254,0.047254,0.039873,0.021893,0.025227,0.025075,-0.017912,-0.059585,-0.046740
4,-0.006994,0.003426,0.006382,0.007610,0.003316,-0.000167,0.010016,-0.009471,0.008195,0.016842,0.001825,0.005384,-0.011494,0.021778,0.015523,0.011810,0.006193,0.016785,0.014291,0.024628,-0.002301,-0.002301,

Now, you insert the `LABEL` column to the `norm_test_df` DataFrame with the corresponding `LABEL` values.

In [ ]:
# Student Action: Apply the 'insert()' function to add the 'LABEL' column to the 'norm_test_df' DataFrame.
norm_test_df.insert(loc=0, column='LABEL', value=exo_test_df['LABEL'])
# After inserting the 'LABEL' column to the 'norm_test_df' DataFrame, print its first five rows.
norm_test_df.head()

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,FLUX.11,FLUX.12,FLUX.13,FLUX.14,FLUX.15,FLUX.16,FLUX.17,FLUX.18,FLUX.19,FLUX.20,FLUX.21,FLUX.22,FLUX.23,FLUX.24,FLUX.25,FLUX.26,FLUX.27,FLUX.28,FLUX.29,FLUX.30,FLUX.31,FLUX.32,FLUX.33,FLUX.34,FLUX.35,FLUX.36,FLUX.37,FLUX.38,FLUX.39,...,FLUX.3158,FLUX.3159,FLUX.3160,FLUX.3161,FLUX.3162,FLUX.3163,FLUX.3164,FLUX.3165,FLUX.3166,FLUX.3167,FLUX.3168,FLUX.3169,FLUX.3170,FLUX.3171,FLUX.3172,FLUX.3173,FLUX.3174,FLUX.3175,FLUX.3176,FLUX.3177,FLUX.3178,FLUX.3179,FLUX.3180,FLUX.3181,FLUX.3182,FLUX.3183,FLUX.3184,FLUX.3185,FLUX.3186,FLUX.3187,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,0.273347,0.228221,0.196676,0.110003,0.104130,0.088690,0.040926,0.014337,0.013534,-0.052079,-0.054832,-0.068804,-0.079655,-0.016015,-0.066188,-0.064169,-0.075732,-0.029872,-0.039691,-0.056415,-0.014730,0.019521,0.079812,0.184609,0.265570,0.305717,0.340084,0.397851,0.429097,0.492255,0.564521,-0.131709,-0.131709,-0.131709,-0.121087,-0.073988,-0.073140,-0.113012,-0.090988,...,-0.007527,0.026450,-0.017873,-0.055658,0.004151,0.067630,0.087497,-0.054121,-0.054121,-0.011702,0.003531,-0.039003,-0.072428,-0.018217,-0.011473,0.029524,0.034158,0.071645,0.112779,0.197754,0.197754,0.096949,0.110186,0.050676,0.073549,0.068892,0.031703,0.023376,0.050355,0.011882,0.031635,0.042578,0.031451,-0.005393,0.028904,0.102708,0.071576,0.080408,0.616438,0.130742
1,2,0.394038,0.391480,0.392680,0.390974,0.388955,0.386673,0.386340,0.382364,0.381035,0.374634,0.365595,0.355955,0.344779,0.326513,0.302050,0.288028,0.267479,0.245166,0.225696,0.204367,0.188959,0.166178,0.151080,0.129661,0.115564,0.104075,0.103383,0.102889,0.109362,0.131452,0.162766,0.195175,0.262094,0.262094,0.254659,0.234561,0.217038,0.197559,0.181571,...,-0.249097,-0.321740,-0.356853,-0.260644,-0.169015,-0.194461,-0.203200,-0.047612,-0.047612,-0.059527,-0.082817,-0.103659,-0.128578,-0.147432,-0.171446,-0.182454,-0.195212,-0.200677,-0.197563,0.074878,0.074878,0.088953,0.093556,0.088384,0.073554,0.071405,0.043269,0.022604,-0.002679,-0.018935,-0.047311,-0.075404,-0.092643,-0.118456,-0.134109,-0.150638,-0.164944,-0.171944,-0.166961,-0.148790
2,2,0.648150,0.627582,0.591444,0.519002,0.466046,0.385214,0.340496,0.281192,0.162553,0.119260,0.069748,-0.010734,-0.022744,-0.045385,-0.006923,0.017249,-0.014986,0.051756,0.045126,0.053615,0.059758,0.081073,0.055856,0.015809,-0.007906,-0.107775,-0.171644,-0.239483,-0.306187,-0.338697,-0.107417,-0.107417,-0.107417,-0.107417,-0.122133,-0.102014,-0.068872,-0.013302,0.014956,...,-0.022256,-0.048845,-0.022302,-0.038442,-0.052251,-0.083176,-0.002191,-0.024047,-0.024047,-0.023925,-0.074831,-0.072713,-0.077209,-0.079091,-0.091627,-0.125196,-0.085210,-0.020016,-0.048883,0.244109,0.244109,0.244109,0.118688,0.046208,0.014201,-0.097274,-0.054964,-0.035257,-0.001916,-0.010436,0.018179,-0.034769,-0.032201,-0.041117,-0.057967,-0.128412,-0.067972,-0.119374,-0.023437,0.027941
3,2,-0.232813,-0.233212,-0.238944,-0.235869,-0.208281,-0.220224,-0.222214,-0.208586,-0.197319,-0.188186,-0.196253,-0.200681,-0.173424,-0.181969,-0.185891,-0.183767,-0.184645,-0.182481,-0.186111,-0.177627,-0.188482,-0.170188,-0.154902,-0.142609,-0.145010,-0.157352,-0.143322,-0.143334,-0.130733,-0.128905,-0.111279,-0.097154,-0.081508,-0.068529,-0.067585,-0.071967,-0.063690,-0.062919,-0.044958,...,0.023932,-0.373173,-0.373173,-0.373173,-0.373173,-0.373173,-0.373173,-0.373173,-0.373173,0.014351,0.038435,0.021458,0.032712,0.049598,0.046797,0.033132,0.025102,0.028333,0.007686,0.008762,-0.002312,-0.044482,-0.066509,-0.084833,-0.083919,-0.062776,-0.055758,-0.051376,-0.020587,0.008095,0.056186,0.047254,0.047254,0.039873,0.021893,0.025227,0.025075,-0.017912,-0.059585,-0.046740
4,2,-0.006994,0.003426,0.006382,0.007610,0.003316,-0.000167,0.010016,-0.009471,0.008195,0.016842,0.001825,0.005384,-0.011494,0.021778,0.015523,0.011810,0.006193,0.016785,0.014291,0.024628,-0.002301,-0.002301,0.004214,-0.003167,0.002058,-0

---

#### Activity 2: Transpose Of A DataFrame^

After the data normalisation, we need to apply Fast Fourier Transformation (which we will learn in the next class). To apply Fast Fourier Transformation, we need to follow three steps.

1. Interchange rows and columns with each other so that columns become rows and rows become columns.

2. Apply the `fft()` function on the DataFrame to apply Fast Fourier Transformation. This we will study in the next class.

3. Interchange rows and columns again.

So, now we will first learn how to interchange the rows and columns. The process of interchanging rows and columns is called **transpose**. It's a very simple process. 

To transpose a DataFrame, you can use the `T` keyword.

In [ ]:
# Student Action: Transpose the 'exo_train_df' using the 'T' keyword.
exo_train_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,5047,5048,5049,5050,5051,5052,5053,5054,5055,5056,5057,5058,5059,5060,5061,5062,5063,5064,5065,5066,5067,5068,5069,5070,5071,5072,5073,5074,5075,5076,5077,5078,5079,5080,5081,5082,5083,5084,5085,5086
LABEL,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
FLUX.1,93.85,-38.88,532.64,326.52,-1107.21,211.10,9.34,238.77,-103.54,-265.91,118.81,-239.88,70.34,424.14,-267.21,35.92,-122.30,-65.20,-66.47,560.19,-1831.31,2053.62,-48.48,145.84,207.37,304.50,150725.80,124.39,-63.50,31.29,-472.50,194.82,26.96,43.07,-248.23,22.82,26.24,-141.22,-35.62,142.40,...,3.26,5.52,223.80,-2.53,307.00,-354.25,2.10,9.27,-863.62,75.06,6.59,28.79,-83.00,9.80,89.83,159.78,11.50,-29.79,-188.62,4.67,15.62,-20.35,-2.30,3.15,462.33,-6.91,122.63,-1090.87,2.13,5.74,125.57,7.45,475.61,-46.63,299.41,-91.91,989.75,273.39,3.82,323.28
FLUX.2,83.81,-33.83,535.92,347.39,-1112.59,163.57,49.96,262.16,-118.97,-318.59,110.97,-164.28,63.86,407.71,-239.11,45.84,-122.30,-76.33,-15.50,262.94,-1781.44,2126.05,-22.95,137.82,195.04,275.94,129578.36,72.73,-49.15,25.14,-384.09,162.51,38.98,46.73,-243.59,46.37,42.32,-81.79,-28.55,137.03,...,4.11,9.28,223.44,-3.69,289.28,-347.10,9.26,8.92,-595.75,67.38,8.80,27.16,-78.50,21.49,79.95,160.60,8.02,-25.91,-214.18,-1.77,-25.58,-26.38,-11.70,1.68,383.22,-14.21,137.56,-1168.93,1.87,2.19,78.69,10.02,395.50,-55.39,302.77,-92.97,891.01,278.00,2.09,306.36
FLUX.3,20.10,-58.54,513.73,302.35,-1118.95,179.16,33.30,277.80,-108.93,-335.66,79.53,-180.91,58.37,461.59,-233.15,47.99,-131.08,-76.23,-44.59,189.94,-1930.84,2146.33,11.15,96.99,150.45,269.24,102184.98,36.85,-45.99,36.93,-330.42,126.17,25.99,29.43,-217.91,39.61,28.34,-52.28,-27.29,93.65,...,2.25,2.71,217.67,-1.26,191.08,-325.74,1.24,3.91,-425.25,33.94,-3.65,22.20,-83.66,16.65,81.44,186.23,-0.59,-25.00,-233.90,-2.53,-30.91,-21.92,-13.75,-4.79,342.29,0.10,109.06,-1154.31,-4.16,8.66,98.29,6.87,423.61,-64.88,278.68,-78.76,908.53,261.73,-3.29,293.16
FLUX.4,-26.98,-40.09,496.92,298.13,-1095.10,187.82,9.63,190.16,-72.25,-450.47,114.25,-225.69,69.43,428.17,-211.84,74.58,-109.69,-72.58,-49.03,185.12,-2016.72,2159.84,-70.04,17.09,135.34,248.51,82253.98,-4.68,-34.55,16.63,-273.41,129.70,47.28,9.75,-190.69,98.75,24.81,-32.45,-28.94,105.64,...,10.67,15.27,217.86,0.04,181.02,-310.49,6.34,4.28,-369.25,63.81,-11.99,19.47,-77.41,13.00,81.62,179.39,7.03,-25.09,-282.53,-3.44,-41.41,15.82,-6.37,-0.52,257.18,-13.02,135.08,-991.00,0.87,5.31,91.16,-2.82,376.36,-88.75,263.48,-97.33,851.83,236.99,-2.88,287.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FLUX.3193,92.54,0.76,5.06,-12.67,-438.54,19.27,-0.44,95.30,4.53,3162.53,126.14,-215.94,-10.79,138.38,-746.83,20.43,-54.25,31.26,-32.75,-165.00,326.61,2292.64,-4.93,127.98,19.46,-10.73,-13056.11,56.43,-90.38,4.79,-3410.42,-14.98,13.04,13.40,198.05,-5.38,8.49,38.66,-26.96,-39.64,...,-3.61,10.37,-36.74,6.49,7.74,164.87,1.90,-1.87,2918.12,-26.43,11.95,0.30,-58.89,15.28,85.77,-82.07,28.74,13.97,0.63,-13.11,33.58,2.67,1.31,-2.90,11.90,-10.60,-184.60,-488.62,-2.64,6.64,210.09,8.75,163.02,28.82,-74.95,151.75,-136.16,-3.47,-1.50,-25.33
FLUX.3194,39.32,-11.70,-11.80,-8.77,-399.71,-43.90,10.90,48.86,21.95,3398.28,200.36,-293.25,-4.97,190.66,-765.22,22.40,-37.13,21.71,-30.28,7.38,481.28,2454.48,26.74,67.51,9.32,3.49,379.48,-1.95,-74.36,13.26,-3

As you can see, the rows have become columns and columns have become rows.

In the next class, we will apply Fast Fourier Transformation on the DataFrame.

---